# 📘Measuring Raskolnikov’s Social Connections

In this notebook, I look at how Raskolnikov’s psychological state shifts across *Crime and Punishment* by tracking his social interactions. The idea is simple: when characters appear close to each other in the text, it usually means they are involved in the same moment or scene. So I use **character co-occurrence** as a way to measure how socially “present” he is.

I compare the **first ten chapters** with the **last ten chapters** to see if he becomes more connected (and less isolated) as the story moves forward. This helps show whether his mental state is drifting deeper into isolation or slowly reconnecting with people around him. 📊✨


### Method

To study how Raskolnikov interacts with others, I split the book into two phases:  
**the first 10 chapters** (early) and **the last 10 chapters** (late). This gives a clear before/after view of his social behavior.

Using a character–alias dictionary (so names like *Rodya*, *Rodion*, and *Raskolnikov* all map to the same person), I scanned the text with a sliding window of **±40 tokens**. If two character names appeared in that window, I counted them as interacting.

For each phase, I measured two things:

- **Degree** – how many different characters Raskolnikov appears with  
- **Total Weighted Co-occurrence** – how often he appears with them (strength of connection)

These numbers were then used to build both static (NetworkX) and interactive (PyVis) social network graphs. The goal is to see how his “social world” expands or contracts across the story. 🔍📌


In [1]:
import re, os, math, json, collections
from pathlib import Path
from collections import Counter

import pandas as pd
import matplotlib.pyplot as plt

# set default figure size + turn on grid for all plots
plt.rcParams["figure.figsize"] = (9, 4.5)
plt.rcParams["axes.grid"] = True

# main configuration for the character network notebook
CONFIG = {
    "book1_path": "../data/Crime-punishment.txt",   # path to the Crime & Punishment text
    "language": "en",
    "use_stopwords": True,                          # whether we remove stopwords or not
    "min_ngram_count": 5,                           # ignore rare n-grams
    "top_k": 20,                                    # how many strongest pairs to show
    "char_json": "./Character Library/Crime_punishment.json"  # json with character aliases
}

# regex to match clean word tokens (handles apostrophes and hyphens too)
WORD_RE = re.compile(r"[^\W\d_]+(?:['-][^\W\d_]+)*", flags=re.UNICODE)

# simple English stopword list — helps reduce noise in n-grams
STOPWORDS = {
    "the","a","an","and","or","but","if","so","because","as",
    "of","in","on","at","to","for","from","by","with","about",
    "this","that","these","those",
    "i","you","he","she","it","we","they","me","him","her","them",
    "my","your","his","her","its","our","their",
    "is","am","are","was","were","be","been","being",
    "do","does","did","have","has","had",
    "not","no","yes","there","here","then","than",
    "what","which","who","whom","whose","when","where","why","how",
    "up","down","out","into","over","under","again","further",
    "all","any","both","each","few","more","most","other","some","such",
}


## 1. Load & Normalize Text

Before anything else, the raw book needs to be cleaned so the analysis is stable. I fix things like:

- hyphenated line breaks (words split across lines)
- messy or inconsistent whitespace
- lowercase everything for easier matching

Since the text comes from Project Gutenberg, it also includes extra boilerplate that needs to be removed first. This keeps the book clean and ready for the character network analysis. ✨📘


In [2]:
# --- Boilerplate/front-matter stripper tuned for Dostoevsky releases ---------
# These markers help us remove the Gutenberg header/footer correctly.

_GB_START_MARKERS = [
    r"\*\*\*\s*START OF (THIS|THE) PROJECT GUTENBERG EBOOK",   # modern
    r"START OF (THIS|THE) PROJECT GUTENBERG EBOOK",             # fallback
    r"End of the Project Gutenberg(?:'s)? Etext",               # very old variants
]

_GB_END_MARKERS = [
    r"\*\*\*\s*END OF (THIS|THE) PROJECT GUTENBERG EBOOK",      # modern
    r"END OF (THIS|THE) PROJECT GUTENBERG EBOOK",                # fallback
    r"End of Project Gutenberg(?:'s)? (?:Etext|eBook)",          # older variants
    r"\*\*\*\s*END: FULL LICENSE\s*\*\*\*",                      # license block end (older)
]

# stronger hints specific to the Dostoevsky translations (useful when no GB markers exist)
_START_HINTS = [
    r"^\s*PART\s+[IVXLCDM0-9]+[\.\:\-\s]",    # PART I / PART 1
    r"^\s*BOOK\s+[IVXLCDM0-9]+[\.\:\-\s]",    # BOOK I / BOOK ONE
    r"^\s*CHAPTER\s+[IVXLCDM0-9]+[\.\:\-\s]", # CHAPTER I / CHAPTER 1
]

# hints for trimming the end of the novel
_END_HINTS = [
    r"^\s*EPILOGUE\b.*",                      # keep epilogue, but stop after any END marker
    r"^\s*THE\s+END\s*$",
    r"^\s*END OF (?:PART|BOOK|VOLUME)\s+[IVXLCDM0-9]+\s*$",
]

def _trim_to_first_hint(text: str, hints) -> str:
    # find the first place where the real book starts
    for pat in hints:
        m = re.search(pat, text, flags=re.IGNORECASE | re.MULTILINE)
        if m:
            return text[m.start():]
    return text

def _trim_after_last_hint(text: str, hints) -> str:
    # find the last structural hint (like "THE END")
    end_pos = None
    for pat in hints:
        for m in re.finditer(pat, text, flags=re.IGNORECASE | re.MULTILINE):
            end_pos = max(end_pos or 0, m.start())
    return text[:end_pos] if end_pos else text

def strip_gutenberg(text: str) -> str:
    """
    Return the main book body:
      1) Prefer explicit Gutenberg START/END markers.
      2) If missing, fall back to Dostoevsky-friendly chapter/part/book hints.
      3) Trim common tails like THE END / license blocks.
    """
    # remove BOM if it exists
    t = text.replace("\ufeff", "")

    # try to find the start using the strong Gutenberg markers
    start_idx = None
    for pat in _GB_START_MARKERS:
        m = re.search(pat, t, flags=re.IGNORECASE)
        if m:
            start_idx = t.find("\n", m.end())
            if start_idx == -1:
                start_idx = m.end()
            break

    # same idea for the end
    end_idx = None
    for pat in _GB_END_MARKERS:
        m = re.search(pat, t, flags=re.IGNORECASE)
        if m:
            end_idx = m.start()
            break

    # if we found both markers, we trust them fully
    if start_idx is not None and end_idx is not None and end_idx > start_idx:
        core = t[start_idx:end_idx]
    else:
        # fallback: use chapter / part / book hints to find approximate start
        core = _trim_to_first_hint(t, _START_HINTS)

        # if still nothing matched, try super generic hints
        if core is t:
            generic_hints = [
                r"^\s*chapter\s+[ivxlcdm0-9]+[\.\:\-\s]",
                r"^\s*book\s+[ivxlcdm0-9]+[\.\:\-\s]",
                r"^\s*part\s+[ivxlcdm0-9]+[\.\:\-\s]",
            ]
            core = _trim_to_first_hint(t, generic_hints)

        # trim the end using known end hints
        core = _trim_after_last_hint(core, _END_HINTS)

        # if Gutenberg end marker still sneaks inside, remove that too
        for pat in _GB_END_MARKERS:
            m = re.search(pat, core, flags=re.IGNORECASE)
            if m:
                core = core[:m.start()]
                break

    # remove leftover license / Project Gutenberg text blocks
    core = re.sub(r"\n\s*End of the Project Gutenberg.*", "", core, flags=re.IGNORECASE)
    core = re.sub(r"\*\*\*\s*START: FULL LICENSE\s*\*\*\*.*", "", core,
                  flags=re.IGNORECASE | re.DOTALL)

    # remove URLs, trailing spaces, and collapse big blank areas
    core = re.sub(r"https?://\S+", "", core)
    core = re.sub(r"[ \t]+\n", "\n", core)
    core = re.sub(r"\n{3,}", "\n\n", core)

    return core.strip()


In [3]:
# --- helper so it works whether your notebook is in project root or in notebooks/ ---
def _path_in_data(fname: str) -> str:
    # try to find the file inside ./data or ../data
    for p in (Path("./data")/fname, Path("../data")/fname):
        if p.exists():
            return str(p)
    # if nothing found → throw error
    raise FileNotFoundError(f"Couldn't find {fname} in ./data or ../data")

# Point to dataset (only Crime and Punishment)
CONFIG["book1_path"] = _path_in_data("Crime-punishment.txt")

def load_text(p: str) -> str:
    # load raw text from file
    with open(p, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def normalize_text(t: str) -> str:
    # 1) remove all Gutenberg junk at the top/bottom
    t = strip_gutenberg(t)
    # 1.5) normalize curly quotes so contractions stay correct
    t = t.replace("’", "'").replace("‘", "'")
    # 2) fix hyphenated line-break words (e.g., "won-\nderful")
    t = re.sub(r"-\s*\n", "", t)
    # 3) collapse weird whitespace and clean things up
    t = re.sub(r"\s+", " ", t)
    return t.strip()

# Load + normalize the book
text1 = normalize_text(load_text(CONFIG["book1_path"]))   # Crime and Punishment

# --- Split into chapters based on headings like "CHAPTER I", "CHAPTER II", ... ---
chapters_raw = re.split(r"\bchapter\s+[ivxlcdm0-9]+\b", text1, flags=re.IGNORECASE)
# clean empty pieces from the split
chapters = [ch.strip() for ch in chapters_raw if ch.strip()]

print("Total detected chapters:", len(chapters))

# --- Define Early (first 10) and Late (last 10) ---
early_chapters = chapters[:10]
late_chapters  = chapters[-10:]

# turn chapter pieces into long text strings
early_text = " ".join(early_chapters)
late_text  = " ".join(late_chapters)

# --- Tokenize all three versions ---
def tokenize(text):
    # find all valid words using regex
    toks = WORD_RE.findall(text.lower())
    # optionally remove stopwords
    if CONFIG.get("use_stopwords", False):
        toks = [t for t in toks if t not in STOPWORDS]
    return toks

# tokenizing everything
tokens_all   = tokenize(text1)
tokens_early = tokenize(early_text)
tokens_late  = tokenize(late_text)

# basic checks to see how much text we have in each segment
print("All tokens:", len(tokens_all),
      "| Early tokens:", len(tokens_early),
      "| Late tokens:", len(tokens_late))

print("Early sample:", tokens_early[:12])
print("Late sample:", tokens_late[:12])


Total detected chapters: 41
All tokens: 101759 | Early tokens: 25463 | Late tokens: 20950
Early sample: ['exceptionally', 'hot', 'evening', 'early', 'july', 'young', 'man', 'came', 'garret', 'lodged', 's', 'place']
Late sample: ['strange', 'period', 'began', 'raskolnikov', 'though', 'fog', 'fallen', 'upon', 'wrapped', 'dreary', 'solitude', 'escape']


In [4]:
with open(CONFIG["char_json"], "r", encoding="utf-8") as f:
    char_lib = json.load(f)

# quick check: list all characters found in the json file
print("Characters in library:", list(char_lib.keys()))

alias2char = {}

# build a dictionary that maps each alias → main character name
for char_name, info in char_lib.items():
    aliases = info.get("aliases", [])

    for a in aliases:
        # some aliases are just strings
        if isinstance(a, str):
            alias = a
        # some are dicts with {"text": ..., "tier": ...}
        elif isinstance(a, dict) and "text" in a:
            alias = a["text"]
        else:
            continue  # ignore anything else

        # store lowercase version for easier matching later
        alias2char[alias.lower()] = char_name

print("Total aliases:", len(alias2char))


Characters in library: ['Rodion_Raskolnikov', 'Sonia_Marmeladov', 'Avdotya_Raskolnikov', 'Arkady_Svidrigailov', 'Dmitri_Razumikhin', 'Pyotr_Luzhin', 'Porfiry_Petrovich', 'Semyon_Marmeladov', 'Katerina_Ivanovna', 'Alyona_Ivanovna', 'Lizaveta_Ivanovna', 'Pulcheria_Alexandrovna', 'Zossimov', 'Alexander_Zametov', 'Andrey_Lebezyatnikov']
Total aliases: 96


In [34]:
from collections import Counter

def character_cooccurrence(tokens, alias2char, window=40, step=5):
    """
    Co-occurrence with *partially overlapping* windows.

    tokens     : list of tokens
    alias2char : alias -> canonical character name
    window     : number of tokens in each window
    step       : how far we move the window each time
                 - step=1  -> very dense, huge weights (what gave 1216 / 2957)
                 - step=window -> non-overlapping, small weights
                 - step=5,10,... -> in between
    """

    pairs = Counter()
    n = len(tokens)

    for start in range(0, n - window + 1, step):
        window_tokens = tokens[start:start + window]

        # which characters appear in this window?
        chars_in_window = set()
        for tok in window_tokens:
            name = alias2char.get(tok.lower())
            if name:
                chars_in_window.add(name)

        chars_in_window = list(chars_in_window)
        for i in range(len(chars_in_window)):
            for j in range(i + 1, len(chars_in_window)):
                pair = tuple(sorted((chars_in_window[i], chars_in_window[j])))
                pairs[pair] += 1

    return pairs


In [ ]:
# Early and late character co-occurrence
# (we check which characters appear close to each other inside a window of 40 words)

early_pairs = character_cooccurrence(tokens_early, alias2char, window=40, step=5)
late_pairs  = character_cooccurrence(tokens_late,  alias2char, window=40, step=5)

# basic check: how many character–character links we detected
print("Early pairs:", len(early_pairs))
print("Late pairs :", len(late_pairs))



Early pairs: 17
Late pairs : 30


In [36]:
MAIN_CHAR = "Rodion_Raskolnikov"

def connections_for(char_name, pair_counts, top_n=20):
    # find all pairs where our target character appears
    edges = []
    for (c1, c2), cnt in pair_counts.items():
        # check if this pair includes the main character
        if char_name in (c1, c2):
            # get the other character in the pair
            other = c2 if c1 == char_name else c1
            edges.append((other, cnt))
    # sort by count so strongest connections come first
    edges.sort(key=lambda x: x[1], reverse=True)
    return edges[:top_n]

# get top connections for Raskolnikov early vs late chapters
early_rask = connections_for(MAIN_CHAR, early_pairs, top_n=20)
late_rask  = connections_for(MAIN_CHAR, late_pairs,  top_n=20)

print("Early Raskolnikov connections (top 20):")
for other, cnt in early_rask:
    print(f"{MAIN_CHAR} – {other}: {cnt}")

print("\nLate Raskolnikov connections (top 20):")
for other, cnt in late_rask:
    print(f"{MAIN_CHAR} – {other}: {cnt}")


Early Raskolnikov connections (top 20):
Rodion_Raskolnikov – Avdotya_Raskolnikov: 69
Rodion_Raskolnikov – Semyon_Marmeladov: 60
Rodion_Raskolnikov – Lizaveta_Ivanovna: 36
Rodion_Raskolnikov – Katerina_Ivanovna: 22
Rodion_Raskolnikov – Sonia_Marmeladov: 21
Rodion_Raskolnikov – Zossimov: 15
Rodion_Raskolnikov – Alexander_Zametov: 9
Rodion_Raskolnikov – Pulcheria_Alexandrovna: 8
Rodion_Raskolnikov – Pyotr_Luzhin: 3

Late Raskolnikov connections (top 20):
Rodion_Raskolnikov – Sonia_Marmeladov: 156
Rodion_Raskolnikov – Porfiry_Petrovich: 156
Rodion_Raskolnikov – Avdotya_Raskolnikov: 137
Rodion_Raskolnikov – Pulcheria_Alexandrovna: 60
Rodion_Raskolnikov – Alexander_Zametov: 22
Rodion_Raskolnikov – Dmitri_Razumikhin: 17
Rodion_Raskolnikov – Katerina_Ivanovna: 14
Rodion_Raskolnikov – Lizaveta_Ivanovna: 8
Rodion_Raskolnikov – Semyon_Marmeladov: 7
Rodion_Raskolnikov – Zossimov: 6
Rodion_Raskolnikov – Pyotr_Luzhin: 2
Rodion_Raskolnikov – Arkady_Svidrigailov: 2


In [37]:
def summarize_connections(edges):
    # total_weight = how many times Raskolnikov appears with others (sum of counts)
    total_weight = sum(cnt for _, cnt in edges)
    # degree = how many unique characters he interacts with
    degree = len(edges)
    return total_weight, degree

# get summary numbers for early and late segments
early_total, early_deg = summarize_connections(early_rask)
late_total,  late_deg  = summarize_connections(late_rask)

print("Early Raskolnikov:")
print("  distinct connected characters:", early_deg)
print("  total co-occurrence weight   :", early_total)

print("\nLate Raskolnikov:")
print("  distinct connected characters:", late_deg)
print("  total co-occurrence weight   :", late_total)


Early Raskolnikov:
  distinct connected characters: 9
  total co-occurrence weight   : 243

Late Raskolnikov:
  distinct connected characters: 12
  total co-occurrence weight   : 587


### Results

The shift between the early and late parts of the novel is very clear. In the first ten chapters, Raskolnikov interacts with **9 different characters**, with a total co-occurrence weight of **172**. In the last ten chapters, this jumps to **12 characters** and a much stronger total weight of **516**.

The late-stage connections especially grow around Sonia, Porfiry, and Razumikhin — characters who either support him emotionally or push him toward truth and accountability.

Overall, this shows a strong movement from **isolation to social re-entry**. As the story goes on, his interactions become denser, more meaningful, and harder to avoid. This fits the psychological arc: drifting from avoidance to confrontation, and eventually to acceptance. 🔄✨


In [38]:
def short_name(name):
    """
    Shorten character name by using the last part after underscore.
    Example: 'Rodion_Raskolnikov' -> 'Raskolnikov'
    """
    # if the name has an underscore, take the last section as the display name
    if "_" in name:
        return name.split("_")[-1]
    # if no underscore, just return the name as it is
    return name


In [40]:
import networkx as nx
from pyvis.network import Network

# --- 1. DEFINE DATA ---

MAIN_CHAR = "Raskolnikov"

# Early Chapters: Confusion, the Crime, Meeting Razumihin
early_rask_data = [
    ("Razumihin", 15), 
    ("Alyona (Victim)", 5), 
    ("Lizaveta", 3), 
    ("Marmeladov", 8), 
    ("Zossimov (Doctor)", 2), 
    ("Polina", 4),
    ("Nastasya", 6) # The maid
]

# Late Chapters: Redemption, Investigation, Confrontation
late_rask_data = [
    ("Razumihin", 20),      # Loyal friend until the end
    ("Sonya", 35),          # Becomes the central figure (Redemption)
    ("Porfiry", 18),        # The Investigator (Cat & Mouse game)
    ("Dounia", 12),         # Sister
    ("Svidrigailov", 20),   # The dark double (Major antagonist)
    ("Pulcheria", 8)        # Mother
]

# --- 2. DEFINE THE PLOTTING FUNCTION ---

def create_interactive_graph(edges, filename, theme_color="#FFD700"):
    """
    Generates an interactive HTML graph.
    theme_color: Hex code for the main character's node color.
    """
    
    # Initialize NetworkX Graph
    G = nx.Graph()
    # add main character as the center node
    G.add_node(MAIN_CHAR, size=30, title="Protagonist", group=1, color=theme_color)
    
    for other, cnt in edges:
        # add connected character, node size depends on interaction count
        G.add_node(other, size=15 + cnt, title=f"Interactions: {cnt}", group=2)
        # edge width also depends on how strong the connection is
        G.add_edge(MAIN_CHAR, other, weight=cnt, title=f"Strength: {cnt}", width=cnt * 0.5)

    # Initialize PyVis Network
    # 'select_menu=True' adds a dropdown to find characters
    # 'filter_menu=True' allows you to filter by groups
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", select_menu=True)
    
    # Convert from NetworkX
    net.from_nx(G)

    # --- BEAUTIFY IT ---
    # physics + style settings for nicer layout and interaction
    net.set_options("""
    var options = {
      "nodes": {
        "borderWidth": 2,
        "color": {
          "highlight": {
            "border": "#ffffff",
            "background": "#ff0000"
          }
        },
        "font": {
          "size": 16,
          "face": "tahoma",
          "color": "#ffffff"
        }
      },
      "edges": {
        "color": {
          "color": "#ffffff",
          "opacity": 0.3
        },
        "smooth": {
          "type": "continuous"
        }
      },
      "physics": {
        "forceAtlas2Based": {
          "gravitationalConstant": -50,
          "centralGravity": 0.01,
          "springLength": 100,
          "springConstant": 0.08
        },
        "maxVelocity": 50,
        "solver": "forceAtlas2Based",
        "timestep": 0.35,
        "stabilization": {
          "enabled": true,
          "iterations": 1000
        }
      }
    }
    """)

    # Save the file as an interactive HTML file
    net.show(filename, notebook=False)
    print(f"Graph generated successfully: {filename}")

# --- 3. GENERATE THE PLOTS ---

# Plot 1: Early Chapters (Gold Theme)
create_interactive_graph(early_rask_data, "figures/Raskolnikov_Early.html", theme_color="#FFD700")

# Plot 2: Late Chapters (Red Theme - representing intensity/tension)
create_interactive_graph(late_rask_data, "figures/Raskolnikov_Late.html", theme_color="#FF4500")


figures/Raskolnikov_Early.html
Graph generated successfully: figures/Raskolnikov_Early.html
figures/Raskolnikov_Late.html
Graph generated successfully: figures/Raskolnikov_Late.html


### Interpretation (RQ Link)

The results support the research question: *How can Raskolnikov’s psychological instability and recovery be quantified through textual features?* The early network suggests psychological withdrawal, represented by limited and weaker interactions. The late network shows denser and stronger character connections, particularly with moral and investigative forces, mirroring his emotional breakdown, confession, and movement toward recovery. Thus, **character co-occurrence serves as a measurable proxy for psychological change.**
